/* PART 2: SQLite
/* We now want you to jump over to a local instance of the database on your machine. 

Copy and paste the LocalSQLConnection.py script into an empty Jupyter notebook, and run it. 

Make sure that the SQLFiles folder containing thes files is in your working directory, and
that you haven't changed the name of the .db file from 'sqlite\db\pythonsqlite'.

You should see the output from the initial query 'SELECT * FROM FACILITIES'.

Complete the remaining tasks in the Jupyter interface. If you struggle, feel free to go back
to the PHPMyAdmin interface as and when you need to. 

You'll need to paste your query into value of the 'query1' variable and run the code block again to get an output.
 

In [3]:
import sqlite3
from sqlite3 import Error
from google.colab import drive
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main(file_location):
    # database = "sqlite\db\pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(file_location)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
    return conn
 
 
if __name__ == '__main__':
    drive.mount('/content/drive/')
    file_location = '/content/drive/My Drive/Springboard - Data Science Career Track/Case Study - Country Club/SQLFiles Tier 1/sqlite_db_pythonsqlite.db'

    conn = main(file_location)
    



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [7]:
q10_query='''with cost_list as 
(
SELECT name AS Facility,
CASE
WHEN Members.memid =0
THEN Bookings.slots * f.guestcost
ELSE Bookings.slots * f.membercost
END AS cost
FROM Members
JOIN Bookings ON Members.memid = Bookings.memid
JOIN Facilities AS f ON Bookings.facid = f.facid
ORDER BY cost DESC
),
revenue_list as (select Facility, SUM(cost) as Revenue from cost_list group by Facility)
select Facility, Revenue from revenue_list where Revenue < 1000;
'''


def execute_query(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


execute_query(conn, q10_query)

('Pool Table', 270)
('Snooker Table', 240)
('Table Tennis', 180)


/* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

In [10]:
q11_query = '''
SELECT m.surname||', '||m.firstname AS Member, r.surname||', '||r.firstname AS Recommended_by
FROM Members AS m
INNER JOIN Members AS r ON m.recommendedby = r.memid
WHERE m.recommendedby != ''
ORDER BY Member
'''

execute_query(conn, q11_query)

('Bader, Florence', 'Stibbons, Ponder')
('Baker, Anne', 'Stibbons, Ponder')
('Baker, Timothy', 'Farrell, Jemima')
('Boothe, Tim', 'Rownam, Tim')
('Butters, Gerald', 'Smith, Darren')
('Coplin, Joan', 'Baker, Timothy')
('Crumpet, Erica', 'Smith, Tracy')
('Dare, Nancy', 'Joplette, Janice')
('Genting, Matthew', 'Butters, Gerald')
('Hunt, John', 'Purview, Millicent')
('Jones, David', 'Joplette, Janice')
('Jones, Douglas', 'Jones, David')
('Joplette, Janice', 'Smith, Darren')
('Mackenzie, Anna', 'Smith, Darren')
('Owen, Charles', 'Smith, Darren')
('Pinker, David', 'Farrell, Jemima')
('Purview, Millicent', 'Smith, Tracy')
('Rumney, Henrietta', 'Genting, Matthew')
('Sarwin, Ramnaresh', 'Bader, Florence')
('Smith, Jack', 'Smith, Darren')
('Stibbons, Ponder', 'Tracy, Burton')
('Worthington-Smyth, Henry', 'Smith, Tracy')


/* Q12: Find the facilities with their usage by member, but not guests */

In [13]:
q12_query = '''
select f.name as Facility, SUM(b.slots) AS Usage_Slots
FROM Bookings as b
INNER JOIN Facilities as f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY Facility
'''
execute_query(conn, q12_query)

('Badminton Court', 1086)
('Massage Room 1', 884)
('Massage Room 2', 54)
('Pool Table', 856)
('Snooker Table', 860)
('Squash Court', 418)
('Table Tennis', 794)
('Tennis Court 1', 957)
('Tennis Court 2', 882)


/* Q13: Find the facilities usage by month, but not guests */

In [17]:

q13_query = '''
select f.name as Facility, strftime("%Y-%m", starttime) as By_month, SUM(b.slots) AS Usage_Slots
FROM Bookings as b
INNER JOIN Facilities as f ON b.facid = f.facid
INNER JOIN Members AS m ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY strftime("%Y-%m", starttime), Facility
'''
execute_query(conn, q13_query)


('Badminton Court', '2012-07', 165)
('Massage Room 1', '2012-07', 166)
('Massage Room 2', '2012-07', 8)
('Pool Table', '2012-07', 110)
('Snooker Table', '2012-07', 140)
('Squash Court', '2012-07', 50)
('Table Tennis', '2012-07', 98)
('Tennis Court 1', '2012-07', 201)
('Tennis Court 2', '2012-07', 123)
('Badminton Court', '2012-08', 414)
('Massage Room 1', '2012-08', 316)
('Massage Room 2', '2012-08', 18)
('Pool Table', '2012-08', 303)
('Snooker Table', '2012-08', 316)
('Squash Court', '2012-08', 184)
('Table Tennis', '2012-08', 296)
('Tennis Court 1', '2012-08', 339)
('Tennis Court 2', '2012-08', 345)
('Badminton Court', '2012-09', 507)
('Massage Room 1', '2012-09', 402)
('Massage Room 2', '2012-09', 28)
('Pool Table', '2012-09', 443)
('Snooker Table', '2012-09', 404)
('Squash Court', '2012-09', 184)
('Table Tennis', '2012-09', 400)
('Tennis Court 1', '2012-09', 417)
('Tennis Court 2', '2012-09', 414)
